<a href="https://colab.research.google.com/github/marcelAnd02/MachineLearning/blob/master/Text_gen_RNN_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import random
from google.colab import drive

In [0]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
raw_data = pd.read_csv('/content/drive/My Drive/Colab_Notebooks/NLP_Project/PoetryFoundationData.csv')
dataset = raw_data.Poem.str.lower().values.tolist()

In [0]:
dataset_cln = []
kpt = {".", "'", ",", " "} #characters to be kept in the dataset
def clean_data():
  for curr_string in dataset:
    new_string = ""
    for ind, cchar in enumerate(curr_string):
      nw_ch = ""
      if (ord(cchar) >= 97 and ord(cchar) <= 122) or (cchar in kpt):
        nw_ch = cchar
      #if (cchar == "\n" and ind != 2 and ind != len(curr_string) - 1):
        #nw_ch = "\n"
      if cchar == "\n":
        nw_ch = " "
      if (nw_ch == " " and (len(new_string) == 0 or (len(new_string) > 0 and new_string[-1] == " "))):
        nw_ch = ""
      new_string += nw_ch
    if len(new_string) > 130:
      dataset_cln.append(new_string)

In [0]:
clean_data()

In [0]:
#print(dataset_cln[258])

In [0]:
hlp_set = set()
for r in dataset_cln:
  for char in r:
    hlp_set.add(char)
chrs = list(hlp_set)
chrs.sort()

In [0]:
chrs_num = dict((ch, it) for it, ch in enumerate(chrs))
num_chrs = dict((it, ch) for it, ch in enumerate(chrs))

In [0]:
chain_len = 120
chr_sqnces = []
nxt_chr = []
def form_inputdata():
  for curr_string in dataset_cln:
    for beg in range(0, len(curr_string) - chain_len, int(chain_len / 6)):
      curr_list = []
      for it in range(beg, beg + chain_len):
        curr_list.append(chrs_num[curr_string[it]])
      chr_sqnces.append(curr_list)
      nxt_chr.append(chrs_num[curr_string[beg + chain_len]])

In [0]:
form_inputdata()
X = np.zeros((len(chr_sqnces), chain_len, len(chrs)), dtype = np.bool)
Y = np.zeros((len(chr_sqnces), len(chrs)), dtype = np.bool)
for tre_ind, tre in enumerate(chr_sqnces):
  for ch_ind, ch in enumerate(tre):
    X[tre_ind][ch_ind][ch] = 1
    Y[tre_ind][nxt_chr[tre_ind]] = 1

# Model

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
from keras.optimizers import adam
from keras.callbacks import ModelCheckpoint

In [0]:
rnn_model = Sequential(
    [
      LSTM(128, input_shape = (chain_len, len(chrs))),
      Dense(len(chrs), activation = "relu"),
      Dense(len(chrs), activation = "relu"),
      Dense(len(chrs)),
      Activation("softmax"),
    ]
)
optimizer = adam(learning_rate = 0.001)
rnn_model.compile(loss = "categorical_crossentropy", optimizer = optimizer)

In [0]:
rnn_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128)               81408     
_________________________________________________________________
dense_4 (Dense)              (None, 30)                3870      
_________________________________________________________________
dense_5 (Dense)              (None, 30)                930       
_________________________________________________________________
dense_6 (Dense)              (None, 30)                930       
_________________________________________________________________
activation_2 (Activation)    (None, 30)                0         
Total params: 87,138
Trainable params: 87,138
Non-trainable params: 0
_________________________________________________________________


In [0]:
checkpoint_path = "/content/drive/My Drive/Colab_Notebooks/NLP_Project/model2"
model_checkpoint_callback = ModelCheckpoint(
    filepath = checkpoint_path,
    save_weights_only = True,
    save_best_only = True
)

In [0]:
rnn_model.fit(X, Y, batch_size = 128, epochs = 10, callbacks = [model_checkpoint_callback])

Epoch 1/10
869426/869426 [==============================] - 2063s 2ms/step - loss: 2.1259
Epoch 2/10


/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:707: RuntimeWarning: Can save best model only with val_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


869426/869426 [==============================] - 2023s 2ms/step - loss: 1.8660
Epoch 3/10
869426/869426 [==============================] - 2028s 2ms/step - loss: 1.7786
Epoch 4/10
869426/869426 [==============================] - 2006s 2ms/step - loss: 1.7241
Epoch 5/10
869426/869426 [==============================] - 1987s 2ms/step - loss: 1.6874
Epoch 6/10
869426/869426 [==============================] - 1997s 2ms/step - loss: 1.6602
Epoch 7/10
869426/869426 [==============================] - 2005s 2ms/step - loss: 1.6401
Epoch 8/10
869426/869426 [==============================] - 2047s 2ms/step - loss: 1.6230
Epoch 9/10
869426/869426 [==============================] - 2038s 2ms/step - loss: 1.6089
Epoch 10/10
869426/869426 [==============================] - 2037s 2ms/step - loss: 1.5970


In [0]:
rnn_model.save_weights("/content/drive/My Drive/Colab_Notebooks/NLP_Project/model2")

In [0]:
def choose_char(preds, temp = 1.0):
  preds = np.asanyarray(preds).astype("float64")
  preds = np.log(preds) / temp
  preds_exp = np.exp(preds)
  preds = preds_exp / np.sum(preds_exp)
  probs = np.random.multinomial(1, preds, 1)
  return np.argmax(probs)

In [0]:
def generate_txt(lng, divs):
  ind1 = random.randint(0, len(dataset_cln))
  ind2 = random.randint(0, len(dataset_cln[ind1]) - chain_len - 10)
  while ind2 < len(dataset_cln[ind1]) and dataset_cln[ind1][ind2] != ' ':
    ind2 += 1
  ind2 += 1
  prevs_chain = dataset_cln[ind1][ind2: ind2 + chain_len]
  gen_text = prevs_chain
  for it in range(0, lng):
    X_gen = np.zeros((1, chain_len, len(chrs)))
    for i, ch in enumerate(prevs_chain):
      X_gen[0][i][chrs_num[ch]] = 1.
    prediction = rnn_model.predict(X_gen, verbose = 0)[0]
    predicted_ind = choose_char(prediction, divs)
    predicted_char = num_chrs[predicted_ind]
    gen_text += predicted_char
    prevs_chain = prevs_chain[1:] + predicted_char
  return gen_text

In [0]:
print(generate_txt(400, 0.7))